In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from glob import glob
from src.utils import (
    read_image, 
    seperate_teeth_to_tooth_info, 
    tooth_num_to_index, 
    tooth_position_to_index
)
from tqdm import tqdm
from cfg import CFG

from src.model import CNNModel

/zz1236zz/anaconda3/envs/pl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Copy and paste your weight path and `hparams.yaml`

In [11]:
weight = "/zz1236zz/workspace/weights/efficientnet_b0_v2/teeth-epoch=02-val/f1=0.99.ckpt"

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

model = CNNModel.load_from_checkpoint(weight)
model.eval()

CNNModel(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      40, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (bn1): BatchNormAct2d(
            40, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1

### inference example code

- 아래 코드에서는 모든 이빨의 이미지를 담은 teeth 이미지와 각 이빨의 이미지에 대한 결과인 tooth 이미지를 구분함.

- 입력
    1. teeth image
    2. 각 tooth 별 정보
        1. segmentation info -> tooth image
        2. tooth_num
        4. decayed
    3. teeth_position (front, upper, left, right, lower)

- 출력 
    1. teeth_image 에 대한 충치 여부를 계산한 f1-score
    2. tooth_image 에 대한 추이 여부를 계산한 f1-score

In [13]:
def tooth_num_to_one_hot_vec(tooth_num):
    tooth_num_idx = tooth_num_to_index(tooth_num)
    tooth_num_one_hot = F.one_hot(tooth_num_idx, num_classes=32).squeeze(0)
    return tooth_num_one_hot

def tooth_position_to_one_hot_vec(tooth_position):
    tooth_position_idx = tooth_position_to_index(tooth_position)
    tooth_position_one_hot = F.one_hot(tooth_position_idx, num_classes=5).squeeze(0)
    return tooth_position_one_hot

# test

In [14]:
import lightning.pytorch as pl
from src.data import TeethDataModule
from cfg import CFG

data_module = TeethDataModule()

trainer = pl.Trainer(
    devices="auto",
    benchmark=True
)

trainer.test(model=model, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Testing DataLoader 0: 100%|██████████| 3000/3000 [02:26<00:00, 20.51it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test/accuracy         0.9973058700561523
         test/f1            0.9973058700561523
    test/f1_weighted        0.9973409175872803
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/accuracy': 0.9973058700561523,
  'test/f1': 0.9973058700561523,
  'test/f1_weighted': 0.9973409175872803}]

# test dataloader

In [17]:
test_dataloader = data_module.test_dataloader()

with torch.no_grad():
    teeth_confusion_mat = np.zeros((2, 2)) 
    tooth_confusion_mat = np.zeros((2, 2))
    
    for X, Y in tqdm(test_dataloader):
        predicted_teeth_decay = 0 # false -> not decayed
        target_teeth_decay = 0 # false -> not decayed
        
        Y = Y.squeeze((0, 2)) # [1 x B x 1] -> [B]

        Y_hat = model.predict(*X)
    
        Y_pred = torch.argmax(Y_hat, dim=1)
        
        for target, pred in zip(Y, Y_pred):
            target = target.item()
            pred = pred.item()
            
            if target: # if target is True, (decayed)
                target_teeth_decay = 1
            if pred:
                predicted_teeth_decay = 1
            
            tooth_confusion_mat[target, pred] += 1
        teeth_confusion_mat[target_teeth_decay, predicted_teeth_decay] += 1

print(teeth_confusion_mat)
print(tooth_confusion_mat)

  2%|▏         | 45/3000 [00:49<55:33,  1.13s/it]  

# Batch version

In [ ]:
test_dir_path = os.path.join("Dataset", "test_data")
test_json_files = glob(os.path.join(test_dir_path, "json", "*"))

sample_num = 100
sample_test_json_files = np.random.choice(test_json_files, sample_num)

with torch.no_grad():
    teeth_confusion_mat = np.zeros((2, 2)) 
    tooth_confusion_mat = np.zeros((2, 2))
    
    for test_json_file in tqdm(sample_test_json_files):
        teeth_dict_list = seperate_teeth_to_tooth_info(test_json_file, test_dir_path)
        
        predicted_teeth_decay = 0 # false -> not decayed
        target_teeth_decay = 0 # false -> not decayed
        
        tooth_imgs_batch = torch.stack([CFG.val_transforms(tooth_info['teeth_image']) for tooth_info in teeth_dict_list])
        tooth_tooth_num_batch = torch.stack([tooth_num_to_one_hot_vec(tooth_info["teeth_num"]) for tooth_info in teeth_dict_list])
        tooth_tooth_position_batch = torch.stack([tooth_position_to_one_hot_vec(tooth_info["teeth_position"]) for tooth_info in teeth_dict_list])
        target_batch = torch.stack([torch.LongTensor([1]) if tooth_info["target"] else torch.LongTensor([0]) for tooth_info in teeth_dict_list])
        
        tooth_imgs_batch = tooth_imgs_batch.to('cuda')
        tooth_tooth_num_batch = tooth_tooth_num_batch.to('cuda')
        tooth_tooth_position_batch = tooth_tooth_position_batch.to('cuda')
        target_batch = target_batch.to('cuda')
        
        Y_hat = model.predict(tooth_imgs_batch, tooth_tooth_num_batch, tooth_tooth_position_batch)
        Y_pred = torch.argmax(Y_hat, dim=1)
        
        for target, pred in zip(target_batch, Y_pred):
            target = target.item()
            pred = pred.item()
            
            if target: # if target is True, (decayed)
                target_teeth_decay = 1
            if pred:
                predicted_teeth_decay = 1
            
            tooth_confusion_mat[target, pred] += 1
        teeth_confusion_mat[target_teeth_decay, predicted_teeth_decay] += 1

print(f"tooth_confusion_matrix:\n{tooth_confusion_mat}")
print(f"teeth_confusion_matrix:\n{teeth_confusion_mat}")

# Iterative

In [ ]:
test_dir_path = os.path.join("Dataset", "test_data")
test_json_files = glob(os.path.join(test_dir_path, "json", "*"))

sample_num = 200
sample_test_json_files = np.random.choice(test_json_files, sample_num)

with torch.no_grad():
    teeth_confusion_mat = np.zeros((2, 2)) 
    tooth_confusion_mat = np.zeros((2, 2))
    
    for test_json_file in tqdm(sample_test_json_files):
        teeth_dict_list = seperate_teeth_to_tooth_info(test_json_file, test_dir_path)
        
        predicted_teeth_decay = 0 # false -> not decayed
        target_teeth_decay = 0 # false -> not decayed
        
        # inference using tooth image
        for tooth_info in teeth_dict_list:
            tooth_img = tooth_info["teeth_image"]
            tooth_num = tooth_info["teeth_num"]
            tooth_position = tooth_info["teeth_position"]
            decayed = tooth_info["target"] # for tooth (true or false)
            decayed = 1 if decayed == "true" else 0 # (true -> 1, false -> 0)
            
            # 하나의 충치라도 있으면 입력 구강 이미지(teeth image)는 
            # 충치가 있는 이미지로 분류
            if decayed == "true":
                target_teeth_decay = 1
            
            # prepare input tooth image
            tooth_img = CFG.val_transforms(tooth_img) # [3x224x224]
            tooth_img = tooth_img.unsqueeze(0) # [1x3x224x224] batch_size=1
            tooth_img = tooth_img.to('cuda')
            
            # prepare tooth_num_onehot_vec
            tooth_num_one_hot = tooth_num_to_one_hot_vec(tooth_num) # [32]
            tooth_num_one_hot = tooth_num_one_hot.to('cuda')
            # prepare tooth_position_onehot_vec
            tooth_position_one_hot = tooth_position_to_one_hot_vec(tooth_position) # [5]
            tooth_position_one_hot = tooth_position_one_hot.to('cuda')
                
            y_hat = model(tooth_img, tooth_num_one_hot, tooth_position_one_hot)
            cls_hat = torch.argmax(y_hat)
            tooth_decay_predicted = int(cls_hat.item())
            
            # cumsum for tooth_confusion_matrix
            tooth_confusion_mat[decayed, tooth_decay_predicted] += 1
            
            if tooth_decay_predicted == 1: # if any decayed tooth
                predicted_teeth_decay = 1 # then given teeth is classified as decayed 
        
        teeth_confusion_mat[target_teeth_decay, predicted_teeth_decay] += 1


print(f"teeth_confusion_matrix:\n{teeth_confusion_mat}")
print(f"tooth_confusion_matrix:\n{tooth_confusion_mat}")